In [4]:
from dotenv import load_dotenv
import requests
import os
load_dotenv()

MINDSDB_URL = 'http://127.0.0.1:47334/api/sql/query'


In [2]:
def mindsdb_query(url, sql_query):
    headers = {"Content-Type": "application/json"}
    return requests.post(headers=headers, json={"query": sql_query}, url= url)

In [3]:
def create_db(url:str, payload:dict, headers: dict):
    return requests.post(url=url, json=payload, headers=headers)


In [5]:
url = "http://127.0.0.1:47334/api/databases"

payload = {"database": {
        "name": "redshift_datasource",
        "engine": "redshift",
        "parameters": {
            "user": os.environ["DEST_USERNAME"],
            "password": os.environ["DEST_PASS"],
            "host": os.environ["DEST_HOST"],
            "port": 5439,
            "database": os.environ["DEST_DB"]
        }
    }}
headers = {"Content-Type": "application/json"}

res = create_db(url=url, payload=payload, headers=headers)

print(res.text)

{"name": "redshift_datasource", "type": "data", "engine": "redshift", "id": 3}
